
# IBM HR 데이터셋 — 페르소나 부여

IBM HR 데이터셋을 이용해 **직원 페르소나(Persona)** 를 자동으로 할당하는 과정을 단계별로 설명합니다.  

**무엇을 하는가?**
1. **데이터 로드**: CSV 파일(IBM_HR.csv)을 읽습니다.  
2. **간단 전처리**: 분석에 필요한 컬럼이 있는지 확인하고, `OverTime(Yes/No)`를 1/0으로 변환합니다.  
3. **정규화(0~1)**: 숫자 컬럼을 0~1 범위로 바꿔서 서로 다른 스케일을 맞춥니다.  
4. **페르소나 점수 계산**: 문서에 정의된 가중치와 방향(높을수록/낮을수록 위험)을 반영하여 **7개 페르소나(P01~P03, S01~S02, N01~N02)** 의 적합도 점수를 구합니다.  
5. **페르소나 할당**  
   - **결정적 할당(Argmax)**: 가장 점수가 높은 페르소나 1개를 고릅니다.  
   - **확률적 할당(Softmax 샘플링)**: 모든 페르소나 점수를 확률로 바꿔 **무작위로 1개**를 선택합니다(재현 가능하도록 시드 고정).  
6. **저장 및 미리보기**: 결과를 CSV로 저장하고, 일부 컬럼만 골라 미리 보여줍니다.

**입력 데이터 경로**: `./data/IBM_HR.csv`  
**출력 CSV 경로**: `./data/IBM_HR_personas_assigned.csv`


In [3]:

# ------------------------------------------------------
# 0) 라이브러리 임포트 및 경로/상수 설정
# ------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

# 입력/출력 파일 경로
DATA_PATH = Path("./data/IBM_HR.csv")
OUTPUT_PATH = Path("./data/IBM_HR_personas_assigned.csv")

# 확률적 샘플링에 사용되는 소프트맥스 온도(낮을수록 가장 큰 점수에 더 쏠림)
SOFTMAX_TAU = 0.1  # 0.1~1.2 사이에서 조정 추천
RNG_SEED = 42      # 재현 가능성 확보(난수 시드 고정)


In [4]:

# ------------------------------------------------------
# 1) 데이터 로드
# ------------------------------------------------------
# - CSV 파일을 읽습니다. 파일이 없으면 에러를 내고 중단합니다.
# - 원본 보존을 위해 복사본(df_orig)을 따로 만듭니다.
if not DATA_PATH.exists():
    raise FileNotFoundError(f"데이터셋을 찾을 수 없습니다: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df_orig = df.copy()

print("데이터 로드 완료:", df.shape)
df.head(3)  # 상위 3행 미리보기


데이터 로드 완료: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


In [5]:

# ------------------------------------------------------
# 2) 간단 전처리 (필수 컬럼 확인, OverTime 변환, 정규화 준비)
# ------------------------------------------------------

# (1) 분석에 필요한 필수 컬럼이 모두 있는지 확인합니다.
required_cols = [
    # 이진/범주형
    "OverTime",
    # 숫자 컬럼(정규화 대상 및 점수 계산에 사용)
    "JobSatisfaction", "PerformanceRating", "JobInvolvement",
    "YearsAtCompany", "RelationshipSatisfaction",
    "YearsSinceLastPromotion", "TrainingTimesLastYear",
    "TotalWorkingYears", "NumCompaniesWorked",
    "PercentSalaryHike"
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    # 실무에서는 컬럼명 오타/스키마 차이가 흔합니다.
    # 이 에러를 보고 데이터 소스/컬럼명을 점검하세요.
    raise ValueError(f"필수 컬럼이 없습니다: {missing}")

# (2) OverTime: Yes/No -> 1/0 으로 변환
#    - 나중에 가중치를 곱해 점수를 만들 때 숫자형이 유리합니다.
df["OverTime_bin"] = df["OverTime"].map({"Yes": 1, "No": 0})
if df["OverTime_bin"].isna().any():
    # Yes/No 외 값이 있더라도 안전하게 0으로 처리
    df["OverTime_bin"] = (df["OverTime"] == "Yes").astype(int)

# 바로 (df["OverTime"] == "Yes").astype(int)만 쓰면 간단하긴 하지만,
# 데이터 이상치 탐지 관점에선, 먼저 .map()으로 정확 매칭을 시도해 NaN이 생겼는지 확인하면 “예상 외 값이 들어왔다”는 신호를 얻을 수 있음.
# → 운영/고도화 단계에서 데이터 품질 모니터링에 유리.

# (3) 정규화 함수 정의 (Min-Max Scaling, 결과를 0~1 범위로)
#    - 서로 다른 단위/범위를 갖는 숫자들을 공정하게 비교하려면 스케일을 맞춰야 합니다.
#    - 모든 값이 같아 분모가 0이 되는 경우(상수열)는 0으로 반환하여 안정성 확보.
def min_max_norm(series: pd.Series) -> pd.Series:
    s = series.astype(float)
    s_min, s_max = s.min(), s.max()
    if pd.isna(s_min) or pd.isna(s_max) or s_max == s_min:
        return pd.Series(0.0, index=s.index)  # 상수열 또는 결측이면 0으로 반환
    return (s - s_min) / (s_max - s_min)

# (4) 정규화 적용할 컬럼들 매핑(원본명 -> 정규화 컬럼명)
norm_map = {
    "JobSatisfaction": "Norm_JobSatisfaction",
    "PerformanceRating": "Norm_PerformanceRating",
    "JobInvolvement": "Norm_JobInvolvement",
    "YearsAtCompany": "Norm_YearsAtCompany",
    "RelationshipSatisfaction": "Norm_RelationshipSatisfaction",
    "YearsSinceLastPromotion": "Norm_YearsSinceLastPromotion",
    "TrainingTimesLastYear": "Norm_TrainingTimesLastYear",
    "TotalWorkingYears": "Norm_TotalWorkingYears",
    "NumCompaniesWorked": "Norm_NumCompaniesWorked",
    "PercentSalaryHike": "Norm_PercentSalaryHike",
}

# (5) 실제 정규화 수행
for src, dst in norm_map.items():
    df[dst] = min_max_norm(df[src])

print("전처리/정규화 완료")
df[[*list(norm_map.values())[:5], "OverTime_bin"]].head(3)


전처리/정규화 완료


,Norm_JobSatisfaction,Norm_PerformanceRating,Norm_JobInvolvement,Norm_YearsAtCompany,Norm_RelationshipSatisfaction,OverTime_bin
0,1.000000,0.0,0.666667,0.15,0.000000,1
1,0.333333,1.0,0.333333,0.25,1.000000,0
2,0.666667,0.0,0.333333,0.00,0.333333,1



## 3) 페르소나 적합도 점수 계산 방식

아래 7개 페르소나에 대해 **가중합 형태의 점수**를 구합니다.  
정규화된 지표(0~1)를 사용하며, **높을수록 해당 페르소나에 가깝다**는 뜻입니다.

- **P01** Burnout(저성과자/번아웃): 초과근무↑, 직무만족↓, 성과↓, 몰입↓  
    - **OverTime** (가중치 0.4), (+, high) : 번아웃의 직접적인 원인인 '높은 직무 요구'를 나타내는 핵심 지표; 가장 높은 가중치 부여.
    - **JobSatisfaction** (가중치 0.3) , (-, low): 낮은 직무 만족도는 번아웃의 주요 증상으로, '낮은 직무 자원'을 반영.
    - **PerformanceRating** (가중치 0.2) , (-, low): 번아웃은 종종 성과 저하로 이어지므로, 이를 결과적 지표로 포함.
    - **JobInvolvement** (가중치 0.1) (-, low): 업무에 대한 몰입도 저하 역시 번아웃의 특징 중 하나.
- **P02** Onboarding Failure(온보딩 실패): 근속↓, 관계만족↓, 직무만족↓  
    - **YearsAtCompany** (가중치 0.5), (-, low): '신규 입사자'라는 특성을 가장 명확하게 나타내는 지표; 근속 연수가 0에 가까울수록 점수가 높아지도록 역방향 변환하여 가장 높은 가중치 부여.
    - **RelationshipSatisfaction** (가중치 0.3), (-, low): 동료 및 상사와의 관계 만족도가 낮은 것은 '고립' 상태를 나타내는 핵심 지표.
    - **JobSatisfaction** (가중치 0.2), (-, low): 조직 부적응은 일반적으로 낮은 직무 만족도로 이어지므로 보조 지표로 사용.

- **P03** Career Stall(성장 정체): 마지막 승진 후 기간↑, 교육횟수↓, 몰입↓
    - **YearsSinceLastPromotion** (가중치 0.5), (+, high): 마지막 승진 이후 기간이 길수록 성장 정체감을 강하게 시사하므로 가장 높은 가중치를 부여.
    - **TrainingTimesLastYear** (가중치 0.3), (-, low): 지난 1년간 교육 횟수가 적은 것은 성장 기회(핵심 직무 자원) 부족을 의미.
    - **JobInvolvement** (가중치 0.2), (-, low): 경력 정체는 업무 몰입도 저하로 이어지는 경향이 있음.
- **S01** Anchor(앵커/안정): 근속↑, 총경력↑, 이직횟수↓  
    - **YearsAtCompany** (가중치 0.4), (+, high): 현 회사에서의 장기 근속은 '앵커'의 가장 중요한 특징.
    - **TotalWorkingYears** (가중치 0.4), (+, high): 총 경력이 긴 것은 조직 내 안정성과 경험의 깊이를 의미하므로 동일하게 높은 가중치를 부여.
    - **NumCompaniesWorked** (가중치 0.2), (-, low): 근무했던 회사의 수가 적을수록 한 조직에 오래 머무는 성향을 나타냄.
- **S02** Rising Star(라이징 스타): 성과↑, 최근 승진함(기간↓), 인상률↑  
    - **PerformanceRating** (가중치 0.4), (+, high): 높은 성과 등급은 '라이징 스타'의 핵심적인 증거.
    - **YearsSinceLastPromotion** (가중치 0.4), (-, low): 최근에 승진했을수록(값이 낮을수록) 빠르게 성장하고 있음을 의미하므로, 역방향 변환하여 높은 가중치를 부여.
    - **PercentSalaryHike** (가중치 0.2), (+, high): 높은 연봉 인상률은 회사가 개인의 성과와 잠재력을 높이 평가하고 있다는 객관적 지표.
- **N01** Coaster(현상 유지자): 몰입↓, 인상률↓, 성과↓  
    - **JobInvolvement** (가중치 0.5), (-, low): 낮은 업무 몰입도는 '현상 유지' 성향을 가장 잘 보여주는 지표.
    - **PercentSalaryHike** (가중치 0.3), (-, low): 낮은 연봉 인상률은 평균적인 성과와 기여도를 반영하는 경향이 있음.
    - **PerformanceRating** (가중치 0.2), (-, low): 성과 등급이 높지 않은(그러나 낮지도 않은) 상태를 반영하기 위해 낮은 가중치로 포함.
- **N02** Competent Malcontent(유능한 불만자): 성과↑, 관계만족↓ (이 두 가지 상반된 특성의 조합이 이 페르소나의 핵심)
    - **PerformanceRating** (가중치 0.5), (-, low): '유능함'을 나타내는 가장 직접적인 지표로 높은 가중치를 부여.
    - **RelationshipSatisfaction** (가중치 0.5), (-, low): '불만자'의 특성을 나타내는 핵심 지표로, 낮은 관계 만족도에 높은 점수를 부여하기 위해 역방향 변환하여 동일한 가중치 부여.


In [6]:

# ------------------------------------------------------
# 3) 페르소나 적합도 점수 계산 (가중합)
# ------------------------------------------------------

# (참고) 각 항은 0~1 범위입니다.
# - 일반적으로 지표가 "높을수록 해당 페르소나에 더 가깝다"면 Norm_x 사용
# - "낮을수록 더 가깝다"면 (1 - Norm_x) 사용
# - 이진은 OverTime_bin 처럼 0/1 값 그대로 사용

# 가독성을 위해 각 페르소나를 별도 변수로 계산한 뒤 DataFrame으로 합칩니다.

# P01: Burnout (저성과자/번아웃)
P01 = (
    0.4 * df["OverTime_bin"] +
    0.3 * (1 - df["Norm_JobSatisfaction"]) +
    0.2 * (1 - df["Norm_PerformanceRating"]) +
    0.1 * (1 - df["Norm_JobInvolvement"])
)

# P02: Onboarding Failure (온보딩 실패)
P02 = (
    0.5 * (1 - df["Norm_YearsAtCompany"]) +
    0.3 * (1 - df["Norm_RelationshipSatisfaction"]) +
    0.2 * (1 - df["Norm_JobSatisfaction"])
)

# P03: Career Stall (성장 정체)
P03 = (
    0.5 * df["Norm_YearsSinceLastPromotion"] +
    0.3 * (1 - df["Norm_TrainingTimesLastYear"]) +
    0.2 * (1 - df["Norm_JobInvolvement"])
)

# S01: Anchor (앵커/안정)
S01 = (
    0.4 * df["Norm_YearsAtCompany"] +
    0.4 * df["Norm_TotalWorkingYears"] +
    0.2 * (1 - df["Norm_NumCompaniesWorked"])
)

# S02: Rising Star (라이징 스타)
S02 = (
    0.4 * df["Norm_PerformanceRating"] +
    0.4 * (1 - df["Norm_YearsSinceLastPromotion"]) +
    0.2 * df["Norm_PercentSalaryHike"]
)

# N01: Coaster (현상 유지자)
N01 = (
    0.5 * (1 - df["Norm_JobInvolvement"]) +
    0.3 * (1 - df["Norm_PercentSalaryHike"]) +
    0.2 * (1 - df["Norm_PerformanceRating"])
)

# N02: Competent Malcontent (유능한 불만자)
N02 = (
    0.5 * df["Norm_PerformanceRating"] +
    0.5 * (1 - df["Norm_RelationshipSatisfaction"])
)

# 점수 테이블로 결합
scores_df = pd.DataFrame({
    "P01": P01, "P02": P02, "P03": P03,
    "S01": S01, "S02": S02,
    "N01": N01, "N02": N02
}, index=df.index)

scores_df.head(3)


,P01,P02,P03,S01,S02,N01,N02
0,0.633333,0.725000,0.366667,0.162222,0.400000,0.666667,0.500000
1,0.266667,0.508333,0.316667,0.377778,0.944762,0.376190,0.500000
2,0.766667,0.766667,0.283333,0.136667,0.457143,0.747619,0.333333



## 4) 페르소나 할당: 결정적 + 확률적

- **결정적(Argmax)**: 가장 점수가 높은 페르소나 1개를 그대로 선택  
- **확률적(Softmax 샘플링)**: 모든 점수를 확률로 바꾸어 무작위로 1개를 선택  
  - 온도(**τ**)를 낮추면 가장 큰 점수에 더 쏠리고, 높이면 다양성이 커집니다.


In [7]:

# ------------------------------------------------------
# 4) 페르소나 할당
# ------------------------------------------------------

# (1) 결정적 할당: argmax
argmax_code = scores_df.idxmax(axis=1)

# (2) 확률적 할당을 위한 softmax 함수
def softmax(arr, tau=1.0, axis=1):
    # 수치 안정성을 위해 최대값을 빼고 exp를 취합니다.
    # 지수( exp )는 입력이 크면 overflow가 나기 쉽습니다.
    # 각 행에서 최대값을 빼고 지수를 취하면, 가장 큰 값은 exp(0)=1, 나머지는 exp(음수)<1이 되어 안정적입니다.
    m = np.max(arr, axis=axis, keepdims=True)
    ex = np.exp((arr - m) / tau)
    return ex / np.sum(ex, axis=axis, keepdims=True)

probs = softmax(scores_df.values, tau=SOFTMAX_TAU, axis=1)
probs_df = pd.DataFrame(
    probs, columns=[f"Prob_{c}" for c in scores_df.columns], index=df.index
)

# (3) 확률적 샘플링: 확률분포(probs)에 따라 1개 페르소나 코드를 선택
rng = np.random.default_rng(RNG_SEED)  # 재현 가능한 난수 생성기(default_rng()는 NumPy의 난수 생성기인 Generator를 반환)
codes = scores_df.columns.to_list()

def sample_code(prob_row):
    return rng.choice(codes, p=prob_row)  # p=prob_row는 합이 1이어야 함(softmax가 보장)

softmax_codes = np.apply_along_axis(sample_code, 1, probs)

# 라벨(이름/리스크 티어) 매핑
persona_name = {
    "P01": "Burnout (저성과자/번아웃)",
    "P02": "Onboarding Failure (온보딩 실패)",
    "P03": "Career Stall (성장 정체)",
    "S01": "Anchor (앵커)",
    "S02": "Rising Star (라이징 스타)",
    "N01": "Coaster (현상 유지자)",
    "N02": "Competent Malcontent (유능한 불만자)",
}
persona_risk_tier = {
    "P01": "High", "P02": "High", "P03": "High",
    "S01": "Stable", "S02": "Stable",
    "N01": "Neutral", "N02": "Neutral",
}

argmax_name = argmax_code.map(persona_name)
argmax_tier = argmax_code.map(persona_risk_tier)

softmax_names = pd.Series(softmax_codes, index=df.index).map(persona_name)
softmax_tiers = pd.Series(softmax_codes, index=df.index).map(persona_risk_tier)

# 분포 확인(각 페르소나가 차지하는 비율)
print("결정적 할당 분포(비율):")
display(pd.Series(argmax_code).value_counts(normalize=True).rename("Assigned share").sort_index().to_frame())

print("확률적 샘플링 분포(비율):")
display(pd.Series(softmax_codes).value_counts(normalize=True).rename("Sampled share").sort_index().to_frame())


결정적 할당 분포(비율):


,Assigned share
N01,0.332653
N02,0.036735
P01,0.146939
P02,0.333333
P03,0.013605
S01,0.021769
S02,0.114966


확률적 샘플링 분포(비율):


,Sampled share
N01,0.310884
N02,0.043537
P01,0.144898
P02,0.308163
P03,0.024490
S01,0.033333
S02,0.134694



## 5) 결과 저장 및 미리보기

- `scores`(페르소나 점수), `probs`(확률), `argmax_*`(결정적), `softmax_*`(확률적)를 **원본 데이터에 합쳐서** CSV로 저장합니다.


In [8]:

# ------------------------------------------------------
# 5) 저장 및 미리보기
# ------------------------------------------------------

# 최종 결과 테이블 구성
result = df_orig.copy()

# 점수/확률 붙이기
result = pd.concat([
    result,
    scores_df.add_prefix("Score_"),
    probs_df
], axis=1)

# 결정적 & 확률적 결과 붙이기
result["argmax_Persona_Code"] = argmax_code
result["argmax_Persona"] = argmax_name
result["argmax_RiskTier"] = argmax_tier

result["softmax_Persona_Code"] = softmax_codes
result["softmax_Persona"] = softmax_names
result["softmax_RiskTier"] = softmax_tiers

# CSV로 저장
result.to_csv(OUTPUT_PATH, index=False)
print("저장 완료:", OUTPUT_PATH)

# 미리보기(핵심 컬럼 위주)
preview_cols = [
    "OverTime", "JobSatisfaction", "PerformanceRating", "JobInvolvement",
    "YearsAtCompany", "RelationshipSatisfaction", "YearsSinceLastPromotion",
    "TrainingTimesLastYear", "TotalWorkingYears", "NumCompaniesWorked", "PercentSalaryHike",
    "Score_P01", "Score_P02", "Score_P03", "Score_S01", "Score_S02", "Score_N01", "Score_N02",
    "argmax_Persona_Code", "argmax_Persona", "argmax_RiskTier",
    "softmax_Persona_Code", "softmax_Persona", "softmax_RiskTier"
]

result[preview_cols].head(12)


저장 완료: data\IBM_HR_personas_assigned.csv


,OverTime,JobSatisfaction,PerformanceRating,JobInvolvement,YearsAtCompany,RelationshipSatisfaction,YearsSinceLastPromotion,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,...,Score_S01,Score_S02,Score_N01,Score_N02,argmax_Persona_Code,argmax_Persona,argmax_RiskTier,softmax_Persona_Code,softmax_Persona,softmax_RiskTier
0,Yes,4,3,3,6,1,0,0,8,8,...,0.162222,0.400000,0.666667,0.500000,P02,Onboarding Failure (온보딩 실패),High,N01,Coaster (현상 유지자),Neutral
1,No,2,4,2,10,4,1,3,10,1,...,0.377778,0.944762,0.376190,0.500000,S02,Rising Star (라이징 스타),Stable,S02,Rising Star (라이징 스타),Stable
2,Yes,3,3,2,0,2,0,3,7,6,...,0.136667,0.457143,0.747619,0.333333,P01,Burnout (저성과자/번아웃),High,N01,Coaster (현상 유지자),Neutral
3,Yes,3,3,3,8,3,3,3,8,1,...,0.337778,0.320000,0.666667,0.166667,P01,Burnout (저성과자/번아웃),High,S02,Rising Star (라이징 스타),Stable
4,No,2,3,3,2,4,2,3,6,9,...,0.080000,0.360952,0.645238,0.000000,N01,Coaster (현상 유지자),Neutral,P02,Onboarding Failure (온보딩 실패),High
5,No,4,3,3,7,3,3,2,8,0,...,0.350000,0.348571,0.623810,0.166667,N01,Coaster (현상 유지자),Neutral,N01,Coaster (현상 유지자),Neutral
6,Yes,1,4,4,1,1,0,3,12,4,...,0.241111,0.928571,0.107143,1.000000,N02,Competent Malcontent (유능한 불만자),Neutral,N02,Competent Malcontent (유능한 불만자),Neutral
7,No,3,4,3,1,2,0,2,1,1,...,0.197778,0.957143,0.230952,0.833333,S02,Rising Star (라이징 스타),Stable,S02,Rising Star (라이징 스타),Stable
8,No,3,4,2,9,2,1,2,10,0,...,0.390000,0.916190,0.419048,0.833333,S02,Rising Star (라이징 스타),Stable,S02,Rising Star (라이징 스타),Stable
9,No,3,3,3,7,2,7,3,17,6,...,0.306667,0.241905,0.623810,0.333333,P02,Onboarding Failure (온보딩 실패),High,P02,Onboarding Failure (온보딩 실패),High



---

### 추가 팁
- **온도(τ)**: `SOFTMAX_TAU`를 0.1~1.2로 조정하며 확률 분포의 다양성을 조절할 수 있습니다.  
- **가중치 변경**: 특정 조직/도메인에 더 맞게 만들려면 위의 가중치를 조정하세요.  
- **품질 점검**: 
  - `argmax_Persona`의 분포가 특정 집단에 과도하게 쏠리지 않는지 확인  
  - (있다면) 실제 Attrition과의 상관을 보며 가중치를 보정  